# 0. 설정

먼저 몇 개의 모듈을 임포트한다. 맷플롯립 그래프를 인라인으로 출력하도록 만들고 그림을 저장하는 함수를 준비한다. 또한 파이썬 버전이 3.5 이상인지, 사이킷런 버전이 0.20 이상인지도 확인한다.

In [ ]:
# 파이썬 ≥3.5 인지
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 인지
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 1. 데이터 로드

MNIST 데이터 셋을 로드한다.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눈다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

# 2. 훈련

랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM(서포트 벡터 머신), MLP 분류기를 훈련시킨다. 엑스트라 트리 분류기는 랜덤 포레스트와 비슷하지만 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보의 특성을 이용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택한다. 따라서 일반 랜덤 포레스트보다 시간이 덜 소요된다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier #다중 레이블 분류를 위한 MLP 분류기

In [ ]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42) #n_estimators : 트리의 개수
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42) #max_iter 인자를 이용해 학습 반복 횟수를 100으로 설정하였다.
mlp_clf = MLPClassifier(random_state=42)

아래는 각 분류기를 이용한 훈련 과정과 결과이다.

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                 

In [ ]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9692, 0.9715, 0.8662, 0.9659]

순서대로 랜덤 포레스트 , 엑스트라 트리, SVM, MLP 분류기의 성능이다. SVM이 다른 분류기보다 성능이 많이 떨어지는 것을 확인할 수 있다.

그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [ ]:
voting_clf = VotingClassifier(named_estimators)

In [ ]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

성능은 다음과 같이 확인할 수 있다.

In [ ]:
voting_clf.score(X_val, y_val)

0.9702

각 분류기의 테스트셋 정확도를 확인해 보았다

In [ ]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.9692, 0.9715, 0.8662, 0.9639]

SVM 모델을 제거해서 성능이 향상되는지 확인한다.

In [ ]:
voting_clf.set_params(svm_clf=None)  #set_params()를 None으로 지정하면 특정 예측기를  제외시킬 수 있다.

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

예측기 목록을 업데이트 한 뒤, 훈련된 예측기 목록에서도 SVM을 제거한다.

In [ ]:
voting_clf.estimators

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [ ]:
del voting_clf.estimators_[2]

In [ ]:
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 LinearSVC(max_iter=100, random_state=42, tol=20),
 MLPClassifier(random_state=42)]

`VotingClassifier`를 다시 평가해본다.

In [ ]:
voting_clf.score(X_val, y_val)

0.9733

성능이 조금 나아진 것을 확인할 수 있다. (SVM 모델이 성능을 저하시킨 것) 아래는 간접 투표 분류기를 사용한 결과이다.

In [ ]:
voting_clf.voting = "soft" #분류기를 다시 훈련시키지 않아도 voting을 soft로 지정해 간접 투표 분류기를 사용할 수 있다.

In [ ]:
voting_clf.score(X_val, y_val)

0.9707

직접 투표 방식이 간접 투표 방식보다 성능이 좋은 것을 알 수 있다.

테스트 세트로 각 분류기의 성능을 다시 확인한다.

In [ ]:
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9715

In [ ]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.9645, 0.9691, 0.9636]

투표 기반 분류기가 오차를 극소로 감소시키는 것을 확인할 수 있다.